In [15]:
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
import json
from joblib import dump

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def draw_hist_boxplot(df,features_names):
  sns.boxplot(x=features_names, data = df)
  sns.stripplot(x=features_names, data = df, color = '#474646')


def replace_outliers_with_median(df, feature_name):
              Q1 = df[feature_name].quantile(0.25)
              median = df[feature_name].quantile(0.5)
              Q3 = df[feature_name].quantile(0.75)
              IQR = Q3 - Q1
              lower_whisker = Q1 - 1.5 * IQR
              upper_whisker = Q3 + 1.5 * IQR
              df[feature_name] = np.where((df[feature_name] < lower_whisker) |
                                          (df[feature_name] > upper_whisker),
                                          median,
                                          df[feature_name])
            
def columns_without_values(data):
    columns = data.isnull().sum()
    print(columns[columns > 0])

In [4]:
df_numbers = pd.read_json('/content/drive/MyDrive/result.json')

In [5]:
df_numbers.drop(df_numbers[df_numbers.year < 2010].index, inplace = True)

In [6]:
df_numbers.drop(df_numbers[df_numbers.year > 2020].index, inplace = True)

In [7]:
df_numbers = df_numbers.drop(['fuelType','make','transmissionType','model','province','id','brand','brand_id','model_id','color','title','seller_type','fuel_types','bodyType','provinceId'], axis = 1)

In [8]:
target = 'price'
all_features = ['km','year','cubicCapacity','doors','hp']

In [9]:
X_train_full, X_test = train_test_split(df_numbers[all_features + [target]], random_state=1)
X_train, X_val = train_test_split(X_train_full, random_state=1)
y_train = X_train[target].values
y_val = X_val[target].values
y_test = X_test[target].values
X_train.drop([target], axis=1, inplace=True)
X_val.drop([target], axis=1, inplace=True)
X_test.drop([target], axis=1, inplace=True)

In [10]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(all_features)]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [11]:
model = build_model()

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                384       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 4,609
Trainable params: 4,609
Non-trainable params: 0
_________________________________________________________________


In [13]:
example_result = model.predict(X_test)
example_result

array([[ -4015.4585],
       [ -3628.526 ],
       [-13098.598 ],
       [ -2228.852 ],
       [-11959.564 ],
       [-15581.062 ],
       [ -3308.388 ],
       [ -8190.073 ],
       [ -6375.695 ],
       [-46890.492 ],
       [ -9639.4375],
       [-21237.879 ],
       [ -5817.5723],
       [ -6623.661 ],
       [-10750.025 ],
       [-13903.096 ],
       [ -5366.597 ],
       [ -2751.9153],
       [ -5104.864 ],
       [ -5878.131 ],
       [-30235.348 ],
       [-11258.383 ],
       [ -2783.1182],
       [-18789.17  ],
       [ -4352.34  ],
       [ -7307.3096],
       [-19161.02  ],
       [ -5524.9106],
       [-10984.34  ],
       [-11339.04  ],
       [ -4314.417 ],
       [ -9707.426 ],
       [ -4581.6562],
       [ -6374.6436],
       [ -8808.714 ],
       [ -8742.246 ],
       [-12886.943 ],
       [-21317.525 ],
       [-19632.996 ],
       [-24109.795 ],
       [-10821.992 ],
       [-10312.8   ],
       [-20761.598 ],
       [-11108.78  ],
       [ -5015.15  ],
       [-1

In [14]:
model.fit(X_train, y_train, epochs=10,validation_split = 0.2, verbose=0)

In [16]:
dump(model, 'keras.sav')

INFO:tensorflow:Assets written to: ram://e3d6e4ae-9874-40c4-a49c-e49a7f25ee31/assets


['keras.sav']